In [76]:
import pandas as pd

data_pd = pd.read_csv("data.csv")
data_pd = data_pd.reset_index().drop(columns=["index"])

In [77]:
def make_list(x):
    x = x[1:-1].split()
    x = list(map(int, x))
    return x

data_pd["prop_mask"] = data_pd["prop_mask"].apply(lambda x: make_list(x))

In [78]:
def preprocessing_translate(text, mask):
    text_parts = []
    text_part = ""
    c_flag = mask[0]
    parts_type = [c_flag] 
    for i, flag in enumerate(mask):
        if c_flag == flag:
            text_part += text[i]
        else:
            c_flag = flag
            parts_type += [c_flag]
            text_parts += [text_part]
            text_part = ""
    text_parts += [text_part]
    return text_parts, parts_type

In [79]:
from googletrans import Translator

translator= Translator()

def preproccesing_connect(text_parts, parts_type):
    prop_mask = []
    text = []
    for i, text_part in enumerate(text_parts):
        text_part = " ".join(text_part.split())
        rus_text = translator.translate(text_part, dest='ru').text
        text_len = len(rus_text)
        part_type = parts_type[i]
        if part_type == 1 or (part_type == 0 and len(text_parts) == 1):
            prop_mask += [part_type] * text_len
        elif i == 0 or i == len(text_parts) - 1:
            prop_mask += [part_type] * (text_len + 1)
        else:
            prop_mask += [part_type] * (text_len + 2)
        text += [rus_text]
    return prop_mask, " ".join(text)

In [ ]:
new_rus_texts = []
new_prop_mask = []
for i in range(data_pd.shape[0]):
    texts, part_types = preprocessing_translate(data_pd["text"][i], data_pd["prop_mask"][i])
    prop_mask, text = preproccesing_connect(texts, part_types)
    new_rus_texts += [text]
    new_prop_mask += [prop_mask]

techniques = data_pd["techniques"]
new_data = pd.DataFrame(np.array([new_rus_texts, new_prop_mask, techniques], dtype=object).T,
                        columns = ["text", "prop_mask", "technique"])